# Migración de información patrimonio

In [125]:
%pip install pandas
%pip install psycopg2
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Carga de archivo de usuarios

In [126]:
import pandas as pd

In [127]:
data = pd.read_excel("../DATA/ultimos_correlativos.xlsx")

## Separacion segun el tipo de Bien

### Bienes ingresados por OC

In [128]:
data_oc = data[data["compra"].notna()]

In [129]:
filas, columnas = data_oc.shape
print("Bienes ingresados por OC")
print('Columnas',columnas)
print('Filas',filas)

Bienes ingresados por OC
Columnas 124
Filas 1118


### Bienes ingresados por NEA

In [130]:
data_nea =  data[data["nea"].notna()]

In [131]:
filas, columnas  = data_nea.shape
print("Bienes ingresados por NEA")
print('Columnas',columnas)
print('Filas',filas)

Bienes ingresados por NEA
Columnas 124
Filas 596


### Bienes sin NEA ni OC

In [132]:
data_null = data[(data["nea"].isnull()) & (data["compra"].isnull())]

In [133]:
filas, columnas = data_null.shape
print("Bienes sin OC ni NEA")
print('Columnas',columnas)
print('Filas',filas)

Bienes sin OC ni NEA
Columnas 124
Filas 253


In [134]:
data_oc.to_excel("ultimos_correlativos_oc.xlsx")
data_nea.to_excel("ultimos_correlativos_nea.xlsx")
data_null.to_excel("ultimos_correlativos_null.xlsx")

## Busqueda de IDITEM por el nombre del bien

### Creación de funcion para encontrar el iditem padre del bien

In [135]:
import psycopg2
import psycopg2.extras

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="bytsscom_unmsm",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
conn.autocommit = True
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

In [136]:
def get_id_item_by_name(bien_name):
    cur.execute(
    """
        WITH OBTENER_COD_ITEM AS (
            SELECT SUBSTRING(cod_item, 1, LENGTH(cod_item) - 4) AS cod_base
            FROM bytsscom_bytsig.item i
            WHERE i.nomb_item = %s AND i.activo_item = 1
        )
        SELECT
            i.id_item,
            SUBSTRING(i.cod_item, 1, LENGTH(i.cod_item) - 4) AS cod_item,
            i.nomb_item
        FROM bytsscom_bytsig.item i
        WHERE i.cod_item LIKE ((SELECT cod_base FROM OBTENER_COD_ITEM) || '%%')
        AND i.activo_item = 1
        ORDER BY i.cod_item
        LIMIT 1;
    """, (bien_name,))
    return cur.fetchone()

In [137]:
def get_id_item_by_same_name(bien_name,cod_bien):
    bien_name = bien_name.strip()
    cod_bien = cod_bien.strip()
    cur.execute(
    """
    SELECT
        i.id_item,
        SUBSTRING(i.cod_item, 1, LENGTH(i.cod_item) - 4) AS cod_item,
        i.nomb_item,
        i.activo_item
    FROM bytsscom_bytsig.item i
    WHERE i.nomb_item = %s AND SUBSTRING(i.cod_item, 1, LENGTH(i.cod_item) - 4) = %s
    """, (bien_name,cod_bien))
    return cur.fetchone()

In [138]:
def get_id_item_by_codbien(cod_bien):
    cod_bien = cod_bien.strip()
    cur.execute(
    """
        SELECT
            i.id_item,
            SUBSTRING(i.cod_item, 1, LENGTH(i.cod_item) - 4) AS cod_item,
            i.nomb_item,
            i.activo_item
        FROM bytsscom_bytsig.item i
        WHERE i.cod_item LIKE ( %s || '%%')
        ORDER BY i.cod_item
        LIMIT 1;
    """, (cod_bien,))
    return cur.fetchone()

In [139]:
get_id_item_by_same_name("ACORDEON",'39220050')

[119625, '39220050', 'ACORDEON', 1]

In [ ]:
import pandas as pd
from psycopg2 import errors  # Para capturar la excepción CardinalityViolation

def get_item_data(bien_name,cod_bien):
    print("Bien actual: ", bien_name)
    try:
        # Llama a la función que ejecuta la consulta SQL
        resultado = get_id_item_by_same_name(bien_name,cod_bien)
    except errors.CardinalityViolation:
        # Si se produce la violación de cardinalidad, retornamos nulos
        return pd.Series([None, None, None, None,None], index=["id_item", "cod_item", "nomb_item","activo_item","encontrado"])
    except Exception as e:
        # Opcional: manejar otras excepciones
        return pd.Series([None, None, None, None,None], index=["id_item", "cod_item", "nomb_item","activo_item","encontrado"])
    
    # Si no se encontró ningún resultado, retornamos nulos
    if resultado is None:
        resultado2 = get_id_item_by_codbien(cod_bien)
        if resultado2 is None:
            return pd.Series([None, None, None, None,None], index=["id_item", "cod_item", "nomb_item","activo_item","encontrado"])
        return pd.Series(list(resultado2) + ["CODIGO"], index=["id_item", "cod_item", "nomb_item","activo_item","encontrado"])
        
    
    # De lo contrario, devolvemos los datos obtenidos con la indicación de que se encontró
    return pd.Series(list(resultado) + ["NOMBRE"], index=["id_item", "cod_item", "nomb_item","activo_item","encontrado"])


    

### Lectura del excel de OC

In [141]:
data_oc = pd.read_excel("ultimos_correlativos_oc.xlsx")

In [142]:
data_oc

,Unnamed: 0,cruce,filtro,cod_depe,codbien,descripcio,marka,marka2023,estado,fec_act,...,cpbaja,baja,codinterno,ubicacion,situacion,mkcp,invent,semana,marlene,guille
0,1,CORRELA,NaN,25.13.00.00.00,675000090002,ACONDICIONADOR CILINDRICO,SANE,N,B,2024-10-21,...,2024-B25,AE-SAN05,NaN,25.13.00.00.00,USO,SANE,NaN,NaN,NOU17,OKEMPLEA2
1,4,CORRELA,NaN,19.60.14.00.00,462200500441,ACUMULADOR DE ENERGIA - EQUIPO DE UPS,NaN,C,N,2025-01-28,...,NaN,NaN,NaN,19.60.14.00.00,USO,NaN,NaN,NaN,NaN,DEPDIC24
2,5,CORRELA,NaN,12.50.41.00.00,952200250002,ADAPTADOR INALAMBRICO DE USB A HDMI/VGA,N,N,B,2020-01-22,...,NaN,NaN,NaN,12.50.41.00.00,USO,N,NaN,NaN,NaN,GTARBOL6
3,6,CORRELA,NaN,05.03.02.04.02,740800280006,ADAPTADOR INALAMBRICO PARA RED,TR,U,B,2020-10-24,...,NaN,NaN,NaN,05.03.02.04.02,USO,TR,NaN,NaN,NaN,GTARBOL6
4,9,CORRELA,NaN,05.13.00.00.00,742200130004,AGENDA ELECTRONICA,SANE,NaN,R,2023-06-12,...,2023-B03,AM-SAN01,NaN,05.13.00.00.00,USO,SANE,NaN,NaN,S23-12,OKEMPLEA2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,1921,CORRELA,NaN,10.02.04.03.00,536499500041,VITRINA METALICA PARA INSTRUMENTAL QUIRURGICO,U,U,B,2023-09-30,...,NaN,NaN,NaN,10.02.04.00.00,USO,U,NaN,NaN,016-ACP-24,DEPDIC24
1114,1923,CORRELA,CP-22-16,24.03.07.00.00,602299500129,VOLTIMETRO,U,U,B,2022-07-11,...,NaN,NaN,NaN,24.03.07.00.00,USO,U,NaN,NaN,NaN,GTARBOL6
1115,1924,CORRELA,OF-583,17.03.00.00.00,677193000001,YATE DE DEPORTE Y/O RECREO,U,U,R,2019-04-11,...,NaN,NaN,NaN,17.03.00.00.00,USO,U,NaN,NaN,NaN,DEPDIC24
1116,1926,CORRELA,NaN,29.03.07.08.03,392299750009,ZAMPOÑA,U,U,B,2022-04-13,...,NaN,NaN,NaN,29.03.07.08.03,USO,U,NaN,NaN,NaN,GTARBOL6


In [143]:
data_oc[['id_item','cod_item','nomb_item','activo_item','encontrado']] = data_oc.apply(
    lambda row: get_item_data(row['descripcio'], str(row['codbien'])[:-4]),
    axis=1
)

Bien actual:  ACONDICIONADOR CILINDRICO


ValueError: Length of values (5) does not match length of index (4)

In [ ]:
data_oc.to_excel("ultimos_correlativos_oc.xlsx")

## Lectura del excel NEA

In [ ]:
data_nea = pd.read_excel("ultimos_correlativos_nea.xlsx")

In [ ]:
data_nea

,Unnamed: 0,cruce,filtro,cod_depe,codbien,descripcio,marka,marka2023,estado,fec_act,...,cpbaja,baja,codinterno,ubicacion,situacion,mkcp,invent,semana,marlene,guille
0,0,CORRELA,NaN,15.40.04.00.00,602200250027,ACELEROMETRO,UB,U,B,2024-01-10,...,NaN,NaN,NaN,15.40.04.00.00,USO,UB,NaN,NaN,NaN,DEPDIC24
1,3,CORRELA,NaN,12.50.00.00.00,462200250001,ACTIVADOR/DESACTIVADOR DE ETIQUETAS ELECTROMAG...,NaN,C,N,2024-11-20,...,NaN,NaN,NaN,12.50.00.00.00,USO,C,NaN,NaN,NaN,GTARBOL6
2,8,CORRELA,NaN,03.13.08.01.00,952200500063,AFINADOR DE SONIDO,UB,U,B,2023-05-23,...,NaN,NaN,NaN,03.13.08.01.00,USO,UB,NaN,NaN,NaN,GTARBOL6
3,9,CORRELA,NaN,05.13.00.00.00,742200130004,AGENDA ELECTRONICA,SANE,NaN,R,2023-06-12,...,2023-B03,AM-SAN01,NaN,05.13.00.00.00,USO,SANE,NaN,NaN,S23-12,OKEMPLEA2
4,10,CORRELA,NaN,19.50.00.00.00,532201600266,AGITADOR (OTROS),NaN,C,N,2025-01-15,...,NaN,NaN,NaN,19.50.00.00.00,USO,NaN,NaN,NaN,NaN,DEPDIC24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,1906,CORRELA,Y,05.04.04.06.04,112292240080,VINOTERA TERMOELECTRICA,U,C,B,2024-04-11,...,NaN,NaN,NaN,05.04.01.00.00,USO,U,NaN,NaN,NaN,GTARBOL6
592,1914,CORRELA,NaN,17.30.21.00.00,746498480035,VITRINA DE MELAMINA,NaN,C,N,2024-09-12,...,NaN,NaN,NaN,17.30.21.00.00,USO,C,NaN,NaN,NaN,GTARBOL6
593,1917,CORRELA,DA,13.20.02.00.00,746499740021,VITRINA DE METAL Y MELAMINA,U,U,B,2024-12-20,...,NaN,NaN,NaN,13.20.02.00.00,USO,U,UACP24-1,NaN,VISITA03EP,DEPDIC24
594,1918,CORRELA,NaN,17.30.26.00.00,112292900029,VITRINA EXHIBIDORA REFRIGERADA - VISICOOLER,NaN,C,N,2024-09-24,...,NaN,NaN,NaN,17.30.26.00.00,USO,C,NaN,NaN,NaN,DEPDIC24


In [ ]:
data_nea[['id_item','cod_item','nomb_item','activo_item','encontrado']] = data_nea.apply(
    lambda row: get_item_data(row['descripcio'], str(row['codbien'])[:-4]),
    axis=1
)

Bien actual:  ACELEROMETRO
Bien actual:  ACTIVADOR/DESACTIVADOR DE ETIQUETAS ELECTROMAGNETICAS
Bien actual:  AFINADOR DE SONIDO
Bien actual:  AGENDA ELECTRONICA
Bien actual:  AGITADOR (OTROS)
Bien actual:  AGITADOR DE BALANCEO
Bien actual:  AGITADOR DE PLACAS
Bien actual:  AGITADOR DE TUBOS
Bien actual:  AGITADOR MAGNETICO
Bien actual:  AGITADOR ORBITAL - AGITADOR DE ROTACION CIRCULAR
Bien actual:  AGITADOR ORBITAL Y VAIVEN
Bien actual:  ALACENA DE MELAMINA
Bien actual:  ALCOHOLIMETRO
Bien actual:  AMOLADORA
Bien actual:  ANALIZADOR DE CALIDAD DE LECHE
Bien actual:  ANALIZADOR DE CORROSION
Bien actual:  ANALIZADOR DE FIBRA OPTICA - ANALIZADOR DE RED
Bien actual:  ANALIZADOR DE OXIGENO
Bien actual:  ANALIZADOR DE PROTEINAS/NITROGENO
Bien actual:  ANALIZADOR DE RADIOCOMUNICACIONES
Bien actual:  ANALIZADOR GENETICO AUTOMATIZADO
Bien actual:  ANALIZADOR INMUNOQUIMICO
Bien actual:  ANALIZADORES (OTROS)
Bien actual:  ANDADOR DE ALUMINIO TERAPEUTICO
Bien actual:  ANTENA PARABOLICA
Bien actual

In [ ]:
data_nea.to_excel("ultimos_correlativos_nea.xlsx")

## Lectura de Bienes sin OC ni NEA

In [ ]:
data_null = pd.read_excel("ultimos_correlativos_null.xlsx")

In [ ]:
data_null

,Unnamed: 0,cruce,filtro,cod_depe,codbien,descripcio,marka,marka2023,estado,fec_act,...,cpbaja,baja,codinterno,ubicacion,situacion,mkcp,invent,semana,marlene,guille
0,2,CORRELA,NaN,03.13.07.07.00,392200500002,ACORDEON,U,U,R,2022-04-13,...,NaN,NaN,NaN,03.13.07.07.00,USO,U,NaN,NaN,NaN,GTARBOL6
1,7,CORRELA,NaN,17.00.00.00.00,532200500003,AFILADOR AUTOMATICO DE CUCHILLAS,SANE,NaN,M,2012-12-05,...,NaN,NaN,NaN,17.00.00.00.00,NaN,SANE,NaN,NaN,NaN,GTARBOL
2,26,CORRELA,NaN,21.00.00.00.00,742200250001,ALIMENTADOR AUTOMATICO DE TRANSPARENCIAS,DONA,NaN,M,2005-10-26,...,NaN,NaN,NaN,21.00.00.00.00,NaN,DONA,NaN,NaN,NaN,GTARBOL
3,28,CORRELA,NaN,17.44.07.00.00,675000380001,ALINEADOR DE RUEDAS - BALANCEADOR DE RUEDAS,U,U,R,2022-04-13,...,NaN,NaN,NaN,17.44.07.00.00,USO,U,NaN,NaN,NaN,GTARBOL6
4,37,CORRELA,NaN,03.05.00.00.00,675001600019,AMPLIADORA DE FOTOGRAFIAS,DONA,NaN,M,2020-07-28,...,2020-D01,TRIMP,NaN,03.05.00.00.00,NaN,DONA,NaN,NaN,NaN,GTARBOL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1898,CORRELA,NaN,05.15.00.00.00,532299590001,VIBRO MASAJEADOR,DONA,NaN,M,2021-06-24,...,2021-D01,PB-CLIN*,NaN,05.15.00.00.00,NaN,DONA,NaN,NaN,4TO,GTARBOL
249,1908,CORRELA,NaN,03.13.08.03.00,392297200001,VIOLONCELLO,U,U,R,2022-04-13,...,NaN,NaN,NaN,03.13.08.03.00,USO,U,NaN,NaN,NaN,GTARBOL6
250,1910,CORRELA,NaN,22.00.00.00.00,952299500005,VISUALIZADOR - VIEWFINDER,DONA,NaN,M,2013-10-09,...,NaN,NaN,NaN,22.00.00.00.00,NaN,DONA,NaN,NaN,NaN,GTARBOL
251,1922,CORRELA,NaN,17.42.00.00.00,678299500001,VOLQUETE,BAJA,NaN,M,2012-02-08,...,NaN,NaN,NaN,17.42.00.00.00,NaN,BAJA,NaN,NaN,NaN,GTARBOL


In [ ]:
data_null[['id_item','cod_item','nomb_item','activo_item','encontrado']] = data_null.apply(
    lambda row: get_item_data(row['descripcio'], str(row['codbien'])[:-4]),
    axis=1
)

Bien actual:  ACORDEON
Bien actual:  AFILADOR AUTOMATICO DE CUCHILLAS
Bien actual:  ALIMENTADOR AUTOMATICO DE TRANSPARENCIAS
Bien actual:  ALINEADOR DE RUEDAS - BALANCEADOR DE RUEDAS
Bien actual:  AMPLIADORA DE FOTOGRAFIAS
Bien actual:  APARADOR DE MADERA
Bien actual:  ARCHIVADOR DE METAL TIPO KARDEX
Bien actual:  ATRIL MECANICO DE METAL
Bien actual:  BAROGRAFO
Bien actual:  BAUL (MAYOR A 1/4 UIT)
Bien actual:  BIOMBO ANTI RAYOS X
Bien actual:  BOMBA TIPO RELOJ PARA TRASEGAR COMBUSTIBLE O ACEITE
Bien actual:  BORRADOR DE CINTAS
Bien actual:  BORRADOR ELECTRICO
Bien actual:  BUSCA PERSONAS - BEEPER
Bien actual:  CABALLETE DE METAL (MAYOR A 1/4 UIT)
Bien actual:  CAJON DE MADERA PARA MUSICA
Bien actual:  CALENTADOR INFRARROJO
Bien actual:  CAMA DE METAL PLEGABLE
Bien actual:  CAMARA DE NIEBLA SALINA
Bien actual:  CAMARA GAMMA
Bien actual:  CAMILLA DE MADERA
Bien actual:  CAMPANA METALICA
Bien actual:  CAMPANA PARA ANAEROBIOSIS
Bien actual:  CAPACIMETRO
Bien actual:  CARDIO MONITOR - MONI

In [ ]:
data_null.to_excel("ultimos_correlativos_null.xlsx")

In [ ]:
cur.close()
conn.close()

## Creación de insert para los excel

In [ ]:
import pandas as pd

In [ ]:
data_oc = pd.read_excel("ultimos_correlativos_oc.xlsx")
data_nea = pd.read_excel("ultimos_correlativos_nea.xlsx")
data_null = pd.read_excel("ultimos_correlativos_null.xlsx")

### Guardamos los bienes que no tienen iditem

In [ ]:
data_oc_sin_iditem = data_oc[data_oc["id_item"].isnull()]
data_nea_sin_iditem = data_nea[data_nea["id_item"].isnull()]
data_null_sin_iditem = data_null[data_null["id_item"].isnull()]

### Filtramos solo aquellos bienes que tienen un iditem

In [ ]:
data_oc = data_oc[data_oc["id_item"].notna()]
data_nea = data_nea[data_nea["id_item"].notna()]
data_null = data_null[data_null["id_item"].notna()]

In [ ]:
data_oc_sin_iditem.to_excel("bienes_oc_sin_iditem.xlsx")
data_nea_sin_iditem.to_excel("bienes_nea_sin_iditem.xlsx")
data_null_sin_iditem.to_excel("bienes_null_sin_iditem.xlsx")

In [ ]:
def create_insert_patrimonio_bien(row):
   # Función auxiliar que verifica si el valor es nulo o vacío
   def value_or_default(val):
        # Comprueba si el valor es None, NaN o cadena vacía (después de quitar espacios)
        if val is None or (isinstance(val, str) and val.strip() == ""):
            return "NO ESPECIFICA"
        try:
            # Para otros tipos (por ejemplo, pandas.NaT o np.nan)
            if pd.isnull(val):
                return "NO ESPECIFICA"
        except Exception:
            pass
        return val

    # Aseguramos convertir a string para poder extraer la subcadena, en caso de ser numérico
   codigo_str = str(row['codigo'])
    # Extraemos los últimos 4 caracteres
   codigo_conc = codigo_str[-4:]
    
   query = f"""INSERT INTO bytsscom_bytsig.patrimonio_bien (
    id_patrimonio_bien,
    id_patrimonio_registro,
    id_item,
    correlativo,
    marca,
    modelo,
    serie,
    dimension,
    color,
    precio,
    detalle,
    estado_patrimonio_bien,
    id_item_especifico)
   VALUES (
      DEFAULT,
      1::integer,
      {row['id_item']}::integer,
      '{codigo_conc}'::varchar,
      '{value_or_default(row['marca'])}'::varchar,
      '{value_or_default(row['modelo'])}'::varchar,
      '{value_or_default(row['serie'])}'::varchar,
      '{value_or_default(row['dimension'])}'::varchar,
      '{value_or_default(row['color'])}'::varchar,
      {row['valorlibro']}::numeric(19,4),
      '{value_or_default(row['otros2'])}'::text,
      'R'::varchar,
      null::integer
   );"""
   query = query.replace("\n", " ")
   return query


### Creacion de insert para OC

In [ ]:
data_oc["query_sql"] = data_oc.apply(create_insert_patrimonio_bien, axis=1)

In [ ]:
queries = "\n".join(data_oc['query_sql'].tolist())

# Guarda la cadena resultante en un archivo .sql
with open("insert_patrimonio_bienes_oc.sql", "w", encoding="utf-8") as f:
    f.write(queries)

### Creación de insert para NEA

In [ ]:
data_nea["query_sql"] = data_nea.apply(create_insert_patrimonio_bien, axis=1)

In [ ]:
queries = "\n".join(data_nea['query_sql'].tolist())

# Guarda la cadena resultante en un archivo .sql
with open("insert_patrimonio_bienes_nea.sql", "w", encoding="utf-8") as f:
    f.write(queries)

### Creación de insert para registros sin OC ni NEA

In [ ]:
data_null["query_sql"] = data_null.apply(create_insert_patrimonio_bien, axis=1)

In [ ]:
queries = "\n".join(data_nea['query_sql'].tolist())

# Guarda la cadena resultante en un archivo .sql
with open("insert_patrimonio_bienes_null.sql", "w", encoding="utf-8") as f:
    f.write(queries)